Part 1
Install lxml for web scraping

In [9]:
!pip3 install lxml --user

     |████████████████████████████████| 5.4 MB 5.0 MB/s eta 0:00:01     |████████████████████████████▏   | 4.8 MB 5.0 MB/s eta 0:00:01


In [34]:
import pandas as pd

The following cell reads the tabular data from the specified url and converts it into a pandas dataframe

In [61]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_list = pd.read_html(url)
df = pd.DataFrame(df_list[0])
print("Shape before ", df.shape)
df.head()

Shape before  (180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Replace 'Not assigned' neighbourhood with corresponding Borough value

In [62]:
for index, row in df.iterrows():
    if df.iloc[index]['Neighbourhood'] == 'Not assigned':
        df.iloc[index]['Neighbourhood'] = df.iloc[index]['Borough']

print("Shape ", df.shape)
df.head()

Shape  (180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Remove rows with Borough = 'Not assigned'

In [63]:
filtered_df = df[df.Borough != 'Not assigned']
filtered_df = filtered_df.reset_index()
del filtered_df['index']
print("Shape after ", filtered_df.shape)
filtered_df.head(10)

Shape after  (103, 3)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Part 2 - Geo Coordinates

In [43]:
!pip install geocoder

     |████████████████████████████████| 102kB 4.7MB/s ta 0:00:01


In [46]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
for postCode in filtered_df['Postal Code']:
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postCode))
      lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

print(latitude)
print(longitude)

"import geocoder # import geocoder\n\n# initialize your variable to None\nlat_lng_coords = None\n\n# loop until you get the coordinates\nfor postCode in filtered_df['Postal Code']:\n    while(lat_lng_coords is None):\n      g = geocoder.google('{}, Toronto, Ontario'.format(postCode))\n      lat_lng_coords = g.latlng\n\nlatitude = lat_lng_coords[0]\nlongitude = lat_lng_coords[1]\n\nprint(latitude)\nprint(longitude)"

Since the above loop was taking long time, I preferred reading the coordinates from the provided csv file

In [64]:
coordinates_df = pd.DataFrame(pd.read_csv('Geospatial_Coordinates.csv'))
coordinates_df.set_index('Postal Code', inplace = True)
coordinates_df.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [65]:
latlong_df = pd.DataFrame(columns=['Latitude','Longitude'])
filtered_df = filtered_df.append(latlong_df)

for index, row in filtered_df.iterrows():
    postCode = filtered_df.iloc[index]['Postal Code'] 
    filtered_df.iloc[index]['Latitude'] = coordinates_df.loc[postCode]['Latitude']
    filtered_df.iloc[index]['Longitude'] = coordinates_df.loc[postCode]['Longitude']

filtered_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895


Part 3 - Cluster the neighbourhoods

Create a new dataframe where the rows contain the word 'Toronto' in Borough

In [66]:
new_df = filtered_df[filtered_df['Borough'].str.contains("Toronto")]
new_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789
15,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754
19,M4E,East Toronto,The Beaches,43.6764,-79.293


Generate cluster label for each neighbourhood 

In [69]:
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

kclusters = 5

new_df1 = new_df.drop(['Postal Code', 'Borough', 'Neighbourhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(new_df1)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 4, 0, 0, 3, 0, 1], dtype=int32)

In [71]:
# add clustering labels
new_df.insert(0, 'Cluster Labels', kmeans.labels_)
new_df.head() 

,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
4,0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
9,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789
15,0,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754
19,4,M4E,East Toronto,The Beaches,43.6764,-79.293


Plot the map of Toronto with 5 clusters highlighted

In [74]:
from geopy.geocoders import Nominatim
import numpy as np

address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(new_df['Latitude'], new_df['Longitude'], new_df['Neighbourhood'], new_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters